In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
os.chdir('C:\\Users\\steve\\OneDrive\\Data Science MSc\\MSc Project') # make sure we start right

##### Exploring the Data

In [41]:
df = pd.read_csv('THIS_IS_IT.csv')
df.tail()

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
726,226399,3104,Control,M,72,1,MRI,Axial PD-T2 TSE,Original,2/14/2011,DCM,NaN
727,301554,3104,Control,M,73,5,MRI,Axial PD-T2 TSE,Original,4/20/2012,DCM,NaN
728,226366,3102,PD,M,64,1,MRI,Axial PD-T2 TSE,Original,11/23/2010,DCM,NaN
729,378210,3102,PD,M,66,7,MRI,Axial PD-T2 TSE FS,Original,02/06/2013,DCM,NaN
730,353457,3102,PD,M,65,21,MRI,Axial PD-T2 TSE,Original,11/21/2011,DCM,NaN


In [42]:
descriptions_array = df.Description.values
np.unique(descriptions_array, return_counts=True) 
# so most is Axial PD-T2 TSE and Axial PD-T2 TSE FS
# leaves us with either 292 or 356, try to find the difference

(array(['AXIAL PD-T2 TSE FS', 'Axial PD-T2 TSE', 'Axial PD-T2 TSE FS',
        'Axial PD-T2 TSE_AC PC line', 'Axial PD-T2 TSE_AC/PC line',
        'Axial PD-T2-FS TSE', 'T2'], dtype=object),
 array([ 33, 292, 356,   2,  24,  23,   1], dtype=int64))

In [43]:
# looks like a bunch of Axial PD-T2 TSE are mislabelled or slightly different, change these
dscrptn = df['Description'].replace({'Axial PD-T2-FS TSE':'Axial PD-T2 TSE FS', 'AXIAL PD-T2 TSE FS':'Axial PD-T2 TSE FS', 
                           'Axial PD-T2 TSE_AC PC line':'Axial PD-T2 TSE_AC/PC line'})

df['Description'] = dscrptn

In [44]:
np.unique(df.Description.values, return_counts=True) 
# looks about right. Now need to decide on TSE vs TSE FS
# well FS has more data, let's try to find something more substantial though

(array(['Axial PD-T2 TSE', 'Axial PD-T2 TSE FS',
        'Axial PD-T2 TSE_AC/PC line', 'T2'], dtype=object),
 array([292, 412,  26,   1], dtype=int64))

In [45]:
# find an example of a TSE and a TSE FS and try to find the difference

In [46]:
tse_fs_test_id = 377981
tse_nofs_test_id = 226399

##### Changing working directory to access the images

In [48]:
# have to dive into the file system to do that.. (on AWS have it all accessed on the bucketz)
import os

In [56]:
os.getcwd()

'C:\\Users\\steve\\OneDrive\\Data Science MSc\\MSc Project'

In [61]:
os.chdir('C:\\Users\\steve\Downloads')

In [62]:
os.listdir()

['.ipynb_checkpoints',
 'Allianz Sales Checklist - compliance.pdf',
 'Anaconda3-2019.03-Windows-x86_64.exe',
 'CIEC_summer',
 'desktop.ini',
 'G Adventures - Checkout_files',
 'game.of.thrones.s08e06.720p.web.h264-memento[eztv].mkv',
 'gimp-2.10.10-setup.exe',
 'Git-2.21.0-64-bit.exe',
 'GoogleVoiceAndVideoSetup.exe',
 'Hardique Manilal cv.docx',
 'lightworks_v14.5.0_full_64bit_setup.exe',
 'nail-shapes.png',
 'NordVPNSetup.exe',
 "Notes on Trieu's FYP.docx",
 'Pictures - Shortcut.lnk',
 'PPMI_THIS_IS_IT',
 'putty-64bit-0.71-installer.msi',
 'python-3.7.3.exe',
 'Screenshot_2019-04-16 e Vision Exam Timetable.png',
 'TableauDesktop-64bit-2019-1-3.exe',
 'THIS_IS_IT.csv',
 'THIS_IS_IT.zip',
 '_temp_matlab_R2019a_win64']